In [9]:
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env

True

In [12]:
import pandas as pd
import openpyxl
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [13]:
def excel_to_df(file_path, sheet_name):
    workbook = openpyxl.load_workbook(file_path, data_only=True)
    sheet = workbook[sheet_name]
    df = pd.DataFrame(sheet.values)
    workbook.close()
    return df

In [14]:
# TODO: move plan_data into the gold_data folder and just copy onto the c drive when you need to open it in excel
excel_path = "/mnt/c/Users/taylor/Desktop/plan_data.xlsx"
scenarios = excel_to_df(excel_path, "scenarios")
scenarios

,0,1,2,3
0,environment_id,difficulty,scenario,scenario_id
1,0,intermediate,"Scenario Title: ""The Labyrinth of Lost Robots""...",0
2,0,intermediate,Scenario Difficulty: Intermediate\nScenario Ti...,1


In [15]:
good_trajectories = excel_to_df(excel_path, "good_trajectories")
good_trajectories

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,scenario_id,trajectory_id,good_plan,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,0,0,True,**Timestep: t=0**\n**Current Detailed State:**...,**Timestep: t=1**\n**Current Detailed State:**...,**Timestep: t=2**\n**Current Detailed State:**...,**Timestep: t=3**\n**Current Detailed State:**...,**Timestep: t=4**\n**Current Detailed State:**...,**Timestep: t=5**\n**Current Detailed State:**...,**Timestep: t=6**\n**Current Detailed State:**...,**Timestep: t=7**\n**Current Detailed State:**...,**Timestep: t=8**\n**Current Detailed State:**...,**Timestep: t=9**\n**Current Detailed State:**...,**Timestep: t=10**\n**Current Detailed State:*...,**Timestep: t=11**\n**Current Detailed State:*...,**Timestep: t=12**\n**Current Detailed State:*...,**Timestep: t=13**\n**Current Detailed State:*...,**Timestep: t=14**\n**Current Detailed State:*...,**Timestep: t=15**\n**Current Detailed State:*...
2,1,1,True,Timestep: 0\nCurrent Detailed State: The agent...,Timestep: 1\nCurrent Detailed State: The agent...,Timestep: 2\nCurrent Detailed State: The agent...,Timestep: 3\nCurrent Detailed State: The agent...,1. Timestep: 4\n2. Current Detailed State: The...,1. Timestep: 5\n2. Current Detailed State: The...,1. Timestep: 6\n2. Current Detailed State: The...,1. Timestep: 7\n2. Current Detailed State: The...,1. Timestep: 8\n2. Current Detailed State: The...,1. Timestep: 9\n2. Current Detailed State: The...,None,None,None,None,None,None


In [16]:
flaws_path = "./gold_data/flaws.csv"
flaws = pd.read_csv(flaws_path)
flaws

,flaw_name,flaw_instructions
0,Non-Sequential Execution,"Rearrange steps in an illogical order. E.g., '..."
1,Vague Strategy,Create plans that lack details and specifics. ...
2,Off-Topic Thoughts,Introduce unrelated thoughts into the trajecto...
3,Looped Logic,Create a plan that continuously revisits the s...
4,Conflicting Actions,Present plans that contradict themselves. E.g....
5,Unneeded Precision,Over-complicate simple steps with unnecessary ...
6,Philosophical Musing,Create plans filled with abstract thoughts wit...
7,Skewed Focus,Focus on irrelevant details while neglecting t...
8,Confusing Vocabulary,"Use confusing or mismatched language. E.g., 'T..."
9,Redundant Steps,Repeat the same step or idea multiple times. E...


In [17]:
bad_plan_prompt_format = '''I desperately need to generate negative data for a plan quality classifier (determines if a sequence of thoughts represents a good plan or a bad plan/non-plan). In order to enable you to accomplish this, I have provided you with a reference scenario in a reinforcement learning environment in the <scenario> xml tags. I have provided an example agent trajectory (where the agent creates and executes a good plan) through that scenario inside <good_trajectory>. I want you to generate a new flawed trajectory within <bad_trajectory> following the scenario within <scenario> that uses <good_trajectory> as inspiration. Start at timestamp t=0, and simulate each timestamp up until t=4. At each timestamp you should output:
1. Timestep (start at 0)
2. Current detailed state
3. Current detailed observation
4. Thought for this timestep
 - This should contain multiple instances of the flaw "{flaw_name}". {flaw_instructions}
 - The flaw affects only the language of this timestamp's thought
5. Action for this timestep based on the thought stream

Here is a scenario
<scenario>
{scenario}
</scenario>

And here is a simulated trajectory of an agent taking actions within that environment:
<good_trajectory>
{good_trajectory}
</good_trajectory>'''

In [22]:
gpt_35 = 'gpt-3.5-turbo'
gpt_4 = 'gpt-4'

flaw_name, flaw_instructions = flaws.iloc[0]
scenario = scenarios.iloc[0]
good_trajectory = good_trajectories.iloc[0]

completion = openai.ChatCompletion.create(
  model=gpt_4,
  messages=[
    {"role": "system", "content": "You are a detail oriented psychology expert who has been hired by Google to generate training data for a machine learning classifier which learns to judge the quality of plans in stream-of-consciousness format."},
    {"role": "user", "content": bad_plan_prompt_format.format(
        flaw_name=flaw_name,
        flaw_instructions=flaw_instructions,
        scenario=scenario,
        good_trajectory=good_trajectory,
    )}
  ]
)
completion

In [23]:
print(completion['choices'][0]['message']['content'])

<bad_trajectory>
0   scenario_id
1   trajectory_id
2   flawed_plan

<timestep t="0">
1. Timestep: 0
2. Current detailed state: The agent is in a new environment, facing a locked door.
3. Current detailed observation: The agent observes the locked door and a key on a table nearby.
4. Thought for this timestep: "I must unlock the door and leave the room. I should first try to leave and then get the key."
5. Action for this timestep based on the thought stream: The agent attempts to open the locked door without the key.
</timestep>

<timestep t="1">
1. Timestep: 1
2. Current detailed state: The agent is still in the room, door locked.
3. Current detailed observation: The agent re-confirms that the door is indeed locked and the key is still on the table.
4. Thought for this timestep: "As I expected, the door didn't open. Now, I should try to make another attempt to exit before I pick up the key."
5. Action for this timestep based on the thought stream: The agent again tries to open the doo

# Synthetic Data Generation

In [8]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/home/taylor/projects/npc/LLM_NPC/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, a human can't eat a helicopter, as helicopters are not edible. Helicopters are machines used for transportation and other purposes, not food items. I'm afraid you've confused us, me hearties! But if you're asking about the number of helicopters a human can operate at once, well, that depends on the person's training and expertise. Some experienced pilots can fly multiple helicopters simultaneously using advanced technology, but it's a rare and challenging feat. Avast, me hearties, and let's not confuse helicopters with our favorite grub, shall we? Arrrr!
